# 检索增强生成（Retrieval Augmented Generation, RAG）第 1 部分：  
## 构建一个利用你自己的文档来辅助生成回答的应用程序。

### 构建一个 **检索增强生成（RAG）** 应用：第 1 部分  
LLM（大语言模型）所赋能的最强大应用之一是**高级问答（Q&A）聊天机器人**。这类应用能够回答与特定信息源相关的问题，其背后使用的技术被称为 **检索增强生成（Retrieval Augmented Generation, RAG）**。

这是一个多部分教程：

- **第 1 部分（本指南）**：介绍 RAG 基本概念，并逐步演示一个最小可行实现。
- **第 2 部分**：扩展实现功能，支持对话式交互和多步骤检索流程。

本教程将展示如何基于文本数据源构建一个简单的问答应用程序。

---

## 概览

一个典型的 RAG 应用通常包含两个主要组成部分：

#### 1. 索引（Indexing）
这是用于从原始数据源中加载并建立索引的流程，通常在离线状态下执行。

#### 2. 检索与生成（Retrieval and Generation）
即真正的 RAG 流程：在运行时接收用户查询，从索引中检索相关信息，并将这些信息传给模型进行内容生成。

---

### 📌 索引阶段（Indexing）

#### 1. 加载（Load）
首先我们需要加载数据，这一步通常通过 **Document Loaders（文档加载器）** 来完成。

#### 2. 分割（Split）
接下来，使用 **文本分割器（Text Splitters）** 将大块文档切分为较小的片段。这样做有两个好处：
- 更容易进行索引和搜索；
- 适配模型有限的上下文窗口（context window），避免超长文本无法处理。

#### 3. 存储（Store）
我们需要一个地方来存储并索引这些小片段，以便后续可以进行检索。这通常使用 **向量数据库（VectorStore）** 和 **嵌入模型（Embeddings model）** 来实现。

![示例图片](../../assets/imgs/rag_indexing.png)  

### 检索与生成（Retrieval and Generation）

#### 4. 检索（Retrieve）  
当用户提供输入时，系统会使用 **检索器（Retriever）** 从存储中找出与其相关的文档片段（即之前分割并嵌入的“chunks”）。

#### 5. 生成（Generate）  
接下来，**聊天模型（ChatModel）或大语言模型（LLM）** 会基于一个包含用户问题和检索结果的提示词（prompt），生成最终的回答。

![示例图片](../../assets/imgs/rag_retrieval_generation.png) 

在我们完成数据的索引处理后，将使用 **LangGraph** 作为编排框架来实现**检索与生成**的整个流程。

In [2]:
# %pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [1]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    _ = load_dotenv()
except ImportError:
    pass

if not os.environ.get("DASHSCOPE_API_KEY"):
  os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [2]:
from langchain_community.chat_models.tongyi import ChatTongyi

llm  = ChatTongyi(
    streaming=True,
    name="qwen-turbo"
)

In [3]:
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
# 初始化 Qwen Embedding 模型
embeddings = DashScopeEmbeddings(model="text-embedding-v1")  

In [4]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

### 预览

在本指南中，我们将构建一个能够回答**网站内容相关问题**的应用程序。我们使用的具体网页是 [**中小企业出海对接交流会**](https://gyxxh.tj.gov.cn/glllm/gabsycs/gxdtgh/202504/t20250408_6902492.html)，这将使我们能够针对该文章的内容提出各种问题。

我们将通过一个简单的索引流程和 RAG 链来实现这一功能，整个实现仅需大约 **50 行代码** 即可完成。

### 详细解析

接下来，我们将**逐步实现**，帮助你真正理解每一步究竟发生了什么。

### 1. 索引（Indexing）

#### 加载文档（Loading Documents）

我们首先需要加载博客文章的内容。为此，我们可以使用 **DocumentLoaders** 工具。这些工具可以从指定的数据源加载数据，并将其返回为一个 `Document` 对象的列表。

在本例中，我们将使用 **WebBaseLoader**，它通过 `urllib` 从网页 URL 加载 HTML 内容，并使用 **BeautifulSoup** 将 HTML 解析为纯文本。

我们可以通过 `bs_kwargs` 参数自定义 HTML 到文本的解析过程（具体可参考 [BeautifulSoup 文档](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)）。  
在这个例子中，只有包含 `class="view-wrap"` 的 HTML 标签是相关的，因此我们会移除其他所有标签。

In [12]:
# import bs4
# from langchain_community.document_loaders import WebBaseLoader

# # Only keep post title, headers, and content from the full HTML.
# bs4_strainer = bs4.SoupStrainer(class_=("view-wrap"))
# loader = WebBaseLoader(
#     web_paths=("https://gyxxh.tj.gov.cn/glllm/gabsycs/gxdtgh/202504/t20250408_6902492.html",),
#     bs_kwargs={"parse_only": bs4_strainer},
# )
# docs = loader.load()

# assert len(docs) == 1
# print(f"Total characters: {len(docs[0].page_content)}")
from langchain_community.document_loaders import PyPDFLoader

# 指定本地 PDF 文件路径
pdf_path = "animal.pdf"  # 修改为你的文件名
loader = PyPDFLoader(pdf_path)

# 加载 PDF 内容
docs = loader.load()

# 打印内容信息
print(f"共加载 {len(docs)} 页")

共加载 307 页


In [15]:
print(docs[50].page_content[:500])

病诊疗与处方手册
除感染消化道外，有时可转移到支气管、肺脏、皮肤、肾脏和心脏。当散播到支气管
和肺脏时，可出现咳嗽、胸痛和体温升高等。
(二)治疗方案
治疗原则为抗真菌、消除病原。
【处方1】克霉唑 15～25毫克/千克，内服，每日2次。
【处方2】伊曲康唑 5～10毫克/千克，内服，每日1～2次，连用2～4个月。
【处方3】制霉菌素、克念菌素、两性霉素 B 和1%碘液外用，每日2～3次，连用
1～2周。
八、芽生菌病
( 一)临床症状
芽生菌病是由皮炎芽生菌引起的一种深部真菌性疾病，主要感染犬、猫的肺脏、皮肤
和消化道。本病潜伏期的长短取决于动物的抵抗力，短的数日或数月，长的数年，多数呈
慢性经过。芽生菌的靶器官多数是肺、眼、皮肤、皮下组织、淋巴结、胃、鼻腔、睾丸和
脑等。这些器官受侵害后出现相应的临床症状，如呼吸困难、咳嗽，X 射线检查发现肺叶
有局限性小结节及纵隔淋巴结肿大等，还可见体温升高、消瘦。有的病例皮肤有溃疡，病
灶伴有渗出物。部分病例出现眼睑肿胀、流泪，有分泌物流出，角膜浑浊，严重的可失
明。如侵害关节、骨骼，则出现跛行。约40%～60%的感染犬表现为弥散性淋巴结病，可


### 深入理解

---

#### 文档加载器（DocumentLoader）  
**定义**：一个从数据源加载数据的对象，返回的是一个 `Document` 对象的列表。

---

#### 分割文档（Splitting Documents）

我们已经加载的文档长度超过 **42,000 个字符**，这远远超出了许多语言模型上下文窗口的限制。即使某些模型能够容纳整篇内容，在非常长的输入中查找信息仍然会变得困难。

为了解决这个问题，我们将整个文档分割成多个小块（chunks），以便进行嵌入（embedding）和向量存储（vector storage）。这样在运行时检索时，系统只需获取与用户问题最相关的部分内容。

我们采用 **RecursiveCharacterTextSplitter**。该工具会递归地尝试使用常见的分隔符（如换行符、空格等）对文档进行切分，直到每个文本块（chunk）的大小符合指定要求为止。

这是推荐用于通用文本场景的文本分割器。

---

### 示例代码：

```python
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # 每个文本块的最大字符数
    chunk_overlap=150  # 块之间的重叠字符数，保持上下文连贯性
)

docs = text_splitter.split_documents(documents)
```

通过这种方式，我们可以将一篇长文章拆分成多个较小的段落，便于后续处理和高效检索。


In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 474 sub-documents.


### TextSplitter（文本分割器）：  
**定义**：一个将 `Document` 列表拆分为更小文本块（chunks）的对象。它是 `DocumentTransformer` 的子类。

---

### DocumentTransformer（文档转换器）：  
**定义**：对 `Document` 对象列表执行某种转换操作的对象。

---

### 存储文档（Storing Documents）

现在我们需要对我们切分后的 **66 个文本块**建立索引，以便在运行时可以对其进行搜索。

我们的方法是：

1. 使用嵌入模型（Embeddings model）对每个文本块的内容进行向量化；
2. 将这些向量插入到一个 **向量数据库（Vector Store）** 中。

这样，当用户提供查询时，我们就可以使用**向量相似性搜索**来检索最相关的文档内容。

---

### 实现方式

我们可以使用在本教程开始时选择的 **向量数据库（VectorStore）** 和 **嵌入模型（Embeddings Model）**，通过一条命令完成所有文档片段的嵌入与存储。

示例代码如下：

```python
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings()
)
```

这段代码会自动为每一个文本块生成嵌入向量，并将其保存到向量数据库中，供后续检索使用。



In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])



['311036f6-b244-422d-bc5f-a7c6dfd5eeea', 'f08a9da2-1c77-4c54-91ff-23c577608aed', '9e0070da-52a9-4a9c-8aea-606ea9c589c7', '4e08b594-559d-4a7f-a6cb-756da27e347d', '1094d584-5dbc-47ee-996c-965f52a2fe3d', '554f7f19-3873-47f6-a4b1-c92e317c56a4', 'b76cc139-cc5d-4dd1-aa56-085e7172dfd4', '3aa553bf-efe7-4aca-ba83-7cd8eecaace5', '2eab7e57-2453-408b-919b-e1be9188b816', '72d1dfe7-3195-45ed-b92a-bbd9f2e95e16', '2f35e52b-25e9-40e5-9ab2-e45eff03da88', '093988ac-8f97-49f1-b1a6-d1deb054f4e2', '438d545d-fb70-45ca-9cd0-f17107e85e6f', '562a566b-b452-42cf-b220-8a94b376f987', '3905603d-ae66-47c0-9369-ba3028af362b', '9e440e4a-d9b0-4818-9097-0e059fa057cb', '0779f739-b461-4c9a-a178-b0e8442bd26b', '3f912ba9-34af-4c3b-a116-5cd38d55ea6f', '03795c3b-b9e4-4b07-bf54-414e7fe9c0a7', '41deab57-316c-43c1-ac6e-53dd8bb7be37', '0bfd0577-c3e5-4c41-be15-3c4461bd1ea9', 'eb9b8329-1a30-4d13-8963-e70aaa816b0b', '961ecd95-b663-4c37-996e-98e79afbb14a', '6e3dfaaa-679c-4cfb-b10e-aefd46b6d3ad', 'e7533c42-c567-4095-917f-c63da27c5347',

### 嵌入模型（Embeddings）  
**定义**：用于将文本转换为嵌入向量（embeddings）的模型封装器（wrapper）。

---

### 向量数据库（VectorStore）  
**定义**：对向量数据库的封装，用于存储和查询嵌入向量。

---

至此，我们完成了整个**索引阶段（Indexing）**。现在我们已经建立了一个可以被查询的向量数据库，其中包含我们博客文章的分块内容。当用户提供一个问题时，理想情况下我们可以返回博客中能够回答该问题的相关片段。

---

## 2. 检索与生成（Retrieval and Generation）

现在我们来编写实际的应用逻辑。

我们希望构建一个简单的应用程序，它能够：

1. 接收用户的提问；
2. 根据问题搜索相关的文档内容；
3. 将检索到的文档内容和原始问题一起传给语言模型；
4. 最终返回一个结构清晰、准确的回答。

在**生成（Generate）**阶段，我们将使用本教程开始时所选择的 **聊天模型（Chat Model）** 来生成最终答案。



In [35]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

(context goes here)

Question: (question goes here)

Helpful Answer:


我们将使用 **LangGraph** 将检索（retrieval）和生成（generation）步骤整合到一个应用程序中。这样做可以带来以下诸多优势：

- 我们可以**一次性定义应用逻辑**，并自动支持多种调用模式，包括流式传输（streaming）、异步调用（async）和批量调用（batched calls）。
- 通过 **LangGraph 平台**，我们可以实现更高效的部署。
- **LangSmith** 将自动追踪应用程序的各个步骤，便于调试与优化。
- 我们可以**轻松地为应用添加关键功能**，例如状态持久化（persistence）和人工审核（human-in-the-loop approval），而且几乎不需要大幅修改代码。

---

### 使用 LangGraph 需要定义三个核心部分：

#### 1. **State（状态）**
控制应用程序的输入数据、各步骤之间的数据传递以及输出结果。  
它通常是一个 `TypedDict`，也可以是一个 `Pydantic BaseModel`。

#### 2. **Nodes（节点）**
代表应用程序中的各个步骤（即函数或操作），例如：加载问题、检索文档、生成答案等。

#### 3. **Control Flow（控制流）**
定义这些节点的执行顺序，比如“先检索再生成”。

---

### State（状态）详解

对于一个简单的 RAG 应用程序，我们只需要跟踪以下三项数据：

```python
from typing import TypedDict

class State(TypedDict):
    question: str      # 用户的问题
    context: str       # 检索到的相关内容
    answer: str        # 最终生成的答案
```

这个状态对象将在整个流程中贯穿始终，作为每一步操作的数据输入和输出载体。


In [23]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

### 节点（Nodes）——应用程序的各个步骤

我们先从一个包含两个步骤的简单流程开始：**检索（retrieval）** 和 **生成（generation）**。

这两个步骤分别对应：

1. **检索（Retrieval）**：根据用户的问题，从向量数据库中查找相关的文档内容。
2. **生成（Generation）**：将检索到的内容和原始问题一起输入给语言模型（LLM），由它生成最终的回答。

在 LangGraph 中，每个步骤都被定义为一个“节点”（Node），这些节点是函数形式的组件，接收当前的状态（State），并返回更新后的状态。通过这种方式，我们可以清晰地组织整个应用的逻辑流程。



In [24]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

### 我们的检索与生成步骤

#### 检索步骤（Retrieval Step）  
我们的检索步骤非常简单：使用用户的输入问题，在向量数据库中执行**相似性搜索**，找到最相关的文档内容（context）。

#### 生成步骤（Generation Step）  
生成步骤则将检索到的上下文和原始问题一起格式化为一个提示词（prompt），然后传给聊天模型（Chat Model），由模型生成自然语言的回答。

---

### 控制流（Control Flow）

最后，我们将整个应用程序编译成一个 **Graph（图）对象**。  
在本例中，我们只是将“检索”和“生成”两个步骤按顺序连接起来，形成一个简单的流程：

```
[Retrieval] → [Generation]
```

这样，LangGraph 就可以根据我们定义的状态（State）、节点（Nodes）和控制流（Control Flow），自动管理数据在各个步骤之间的传递和更新。

In [25]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [28]:
print(graph.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+----------+   
| retrieve |   
+----------+   
      *        
      *        
      *        
+----------+   
| generate |   
+----------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


### 使用（Usage）

让我们来测试一下我们构建的应用程序！  
**LangGraph 支持多种调用方式**，包括：

- 同步调用（sync）
- 异步调用（async）
- 流式调用（streaming）

#### 调用（Invoke）示例：

你可以使用 `.invoke()` 方法以同步方式运行你的 RAG 应用：

```python
response = app.invoke({"question": "文章中提到的自主智能体的核心组件有哪些？"})
print(response["answer"])
```

这会触发整个流程：

1. 使用问题进行文档检索；
2. 将检索结果和问题传给语言模型；
3. 返回生成的答案。


In [32]:
result = graph.invoke({"question": "有多少家企业参加活动?"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

Context: [Document(id='6a34df01-e4f1-4fc4-bff5-ac409b37b82e', metadata={'source': 'https://gyxxh.tj.gov.cn/glllm/gabsycs/gxdtgh/202504/t20250408_6902492.html'}, page_content='来源：\n          天津市工业和信息化局\n        \n发布时间：\n          2025-04-08 09:30\n        \n\n\n为深入贯彻党的二十届三中全会精神，落实《工业和信息化部办公厅关于开展中小企业出海服务专项行动的通知》及《工业和信息化部等17部门办公厅（室）关于开展2025年“一起益企”中小企业服务行动的通知》要求，进一步提升我市中小企业国际化水平和核心竞争力，助力中小企业开拓国际市场、增强风险防控能力，天津市中小企业服务中心于近日成功举办轻工产业链中小企业出海对接交流会。中国中小企业发展促进中心、天津市贸促会、天津市食品工业协会、天津市自行车电动车行业协会、武清区崔黄口镇商会，以及来自自行车、食品、地毯等领域的生产制造企业和进出口企业等20余家单位代表参加了活动。 \u2002会上，国家中心产业集群发展研究所负责同志围绕中小企业“走出去”万帆耘海行动的实施背景、主要任务及具体进展作了专题报告，深入解读了当前中小企业出海的趋势、政策支持及发展机遇，为参会企业提供了权威的政策指导和行动路径。市贸促会会展服务中心相关负责同志重点介绍了贸促会组织的境外展会项目及我市境外参展补贴政策，并结合热点国家的贸易环境、市场动态及政策导向，为企业提供了详实的出海信息及参展支持服务，助力企业精准把握国际市场机遇。活动主题引起与会企业的高度关注，现场交流、对接气氛热烈。近年来，在国家大力推进构建以国内大循环为主体、国内国际双循环相互促进的新发展格局背景下，越来越多的中小企业将海外业务视为推动企业增长的重要战略。但在全球化经营过程中，中小企业面临诸多挑战，由于缺乏出海的实战经验，境外抗风险能力较低，减缓了我市企业出海的步伐。市中小企业服务中心2024年起已着手为有出海需求的企业搭建资源共享、经验互通的平台；2025年中心启动的“中小企业出海系列活动”涵盖出海对接交流会、国际市场调研、境外展会参

### 流式调用步骤（Stream Steps）：

In [33]:
for step in graph.stream(
    {"question": "有多少家企业参加活动?"}, stream_mode="updates"
):
    print(f"{step}\n\n----------------\n")

{'retrieve': {'context': [Document(id='6a34df01-e4f1-4fc4-bff5-ac409b37b82e', metadata={'source': 'https://gyxxh.tj.gov.cn/glllm/gabsycs/gxdtgh/202504/t20250408_6902492.html'}, page_content='来源：\n          天津市工业和信息化局\n        \n发布时间：\n          2025-04-08 09:30\n        \n\n\n为深入贯彻党的二十届三中全会精神，落实《工业和信息化部办公厅关于开展中小企业出海服务专项行动的通知》及《工业和信息化部等17部门办公厅（室）关于开展2025年“一起益企”中小企业服务行动的通知》要求，进一步提升我市中小企业国际化水平和核心竞争力，助力中小企业开拓国际市场、增强风险防控能力，天津市中小企业服务中心于近日成功举办轻工产业链中小企业出海对接交流会。中国中小企业发展促进中心、天津市贸促会、天津市食品工业协会、天津市自行车电动车行业协会、武清区崔黄口镇商会，以及来自自行车、食品、地毯等领域的生产制造企业和进出口企业等20余家单位代表参加了活动。 \u2002会上，国家中心产业集群发展研究所负责同志围绕中小企业“走出去”万帆耘海行动的实施背景、主要任务及具体进展作了专题报告，深入解读了当前中小企业出海的趋势、政策支持及发展机遇，为参会企业提供了权威的政策指导和行动路径。市贸促会会展服务中心相关负责同志重点介绍了贸促会组织的境外展会项目及我市境外参展补贴政策，并结合热点国家的贸易环境、市场动态及政策导向，为企业提供了详实的出海信息及参展支持服务，助力企业精准把握国际市场机遇。活动主题引起与会企业的高度关注，现场交流、对接气氛热烈。近年来，在国家大力推进构建以国内大循环为主体、国内国际双循环相互促进的新发展格局背景下，越来越多的中小企业将海外业务视为推动企业增长的重要战略。但在全球化经营过程中，中小企业面临诸多挑战，由于缺乏出海的实战经验，境外抗风险能力较低，减缓了我市企业出海的步伐。市中小企业服务中心2024年起已着手为有出海需求的企业搭建资源共享、经验互通的平台；2025年中心启动的“中小企业出海系列活动”涵盖出海对接

### 流式传输 Token（Stream Tokens）：

In [34]:
for message, metadata in graph.stream(
    {"question": "有多少家企业参加活动?"}, stream_mode="messages"
):
    print(message.content, end="|")

约|有|2|0余家企业参加了|此次活动。||

## 完整例子

In [42]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://gyxxh.tj.gov.cn/glllm/gabsycs/gxdtgh/202504/t20250408_6902492.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("view-wrap")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [44]:
response = graph.invoke({"question": "有多少家企业参加活动?"})
print(response["answer"])

约20家企业代表参加了活动。  
thanks for asking!
